In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from utils import *

In [26]:
df = pd.read_csv("ghcnd_hcn/pair_final.csv")
df = df.sample(frac=1).reset_index(drop=True)
category = 'PRCP2'
X_train, X_test, y_train, y_test, map_dict = one_station_split(
            df, 'USC00057936', category)
len(map_dict)

/homes/aqngo/project-aqngo-ellin/utils.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[category] = pd.qcut(df[category], q=45, duplicates='drop').astype('category')
/homes/aqngo/project-aqngo-ellin/utils.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[category] = df[category].cat.codes


18

In [27]:
train_stats = X_train.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
LAT,4404.0,40.488300,2.124764e-12,40.4883,40.4883,40.4883,40.4883,40.4883
LON,4404.0,-106.823300,7.603671e-12,-106.8233,-106.8233,-106.8233,-106.8233,-106.8233
ELEV,4404.0,2092.800000,5.275668e-11,2092.8000,2092.8000,2092.8000,2092.8000,2092.8000
DATE,4404.0,729366.509083,4.043774e+03,722817.0000,725911.5000,729137.0000,732766.5000,737331.0000
PRCP1,4404.0,18.250000,3.650184e+01,0.0000,0.0000,0.0000,18.0000,221.0000
TMAX1,4404.0,118.477066,1.190937e+02,-200.0000,17.0000,100.0000,228.0000,317.0000
TMIN1,4404.0,-51.465259,9.203728e+01,-422.0000,-111.0000,-39.0000,22.0000,139.0000
SNOW1,4404.0,16.815849,4.368446e+01,0.0000,0.0000,0.0000,0.0000,462.0000
SNOWD1,4404.0,195.880563,2.887479e+02,0.0000,0.0000,0.0000,356.0000,1448.0000
TMAX2,4404.0,118.221617,1.191800e+02,-200.0000,17.0000,100.0000,228.0000,317.0000


In [28]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(X_train)
normed_test_data = norm(X_test)

In [80]:
def build_model():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(len(map_dict), activation='softmax')
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [81]:
len(y_train.unique())

18

In [83]:
model = build_model()

EPOCHS = 1000

history = model.fit(normed_train_data.values, y_train.values,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])


Epoch: 0, accuracy:0.6372,  loss:1.4360,  val_accuracy:0.6504,  val_loss:1.1335,  
....................................................................................................
Epoch: 100, accuracy:0.9886,  loss:0.0462,  val_accuracy:0.9841,  val_loss:0.1843,  
....................................................................................................
Epoch: 200, accuracy:0.9935,  loss:0.0232,  val_accuracy:0.9818,  val_loss:0.2163,  
....................................................................................................
Epoch: 300, accuracy:0.9960,  loss:0.0157,  val_accuracy:0.9807,  val_loss:0.2689,  
....................................................................................................
Epoch: 400, accuracy:0.9963,  loss:0.0130,  val_accuracy:0.9784,  val_loss:0.3159,  
....................................................................................................
Epoch: 500, accuracy:0.9997,  loss:0.0035,  val_accuracy:0.9796,  val_l

In [76]:
test_loss, test_acc = model.evaluate(X_test.values,  y_test.values, verbose=2)

1102/1 - 0s - loss: 367486.4286 - accuracy: 0.0181


In [77]:
predictions = model.predict(X_test)
# tf.math.confusion_matrix(
#     y_test,
#     predictions,
#     num_classes=None,
#     weights=None,
#     dtype=tf.dtypes.int32,
#     name=None
# )

In [69]:
y_test.values

array([14,  5,  0, ...,  0, 17,  0], dtype=int8)

In [79]:
model.predict(X_test)[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)